In [2]:
import numpy as np
from scipy.linalg import expm
from itertools import product

# --- Pauli matrices and utilities ---
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

def kron_n(*ops):
    return np.kron(np.kron(np.kron(ops[0], ops[1]), ops[2]), ops[3])

# --- Operators for building H ---
def number_op(k):
    ops = [I] * 4
    ops[k] = 0.5 * (I + Z)
    return kron_n(*ops)

def SP(k):
    ops = [I] * 4
    ops[k] = 0.5 * (X - 1j * Y)
    return kron_n(*ops)

def SM(k):
    ops = [I] * 4
    ops[k] = 0.5 * (X + 1j * Y)
    return kron_n(*ops)

def sigma_anyonic(k):
    ops = [I] * 4
    ops[k] = Z
    return kron_n(*ops)

def jw_string(path):
    if not path:
        return np.eye(16)
    result = sigma_anyonic(path[0])
    for k in path[1:]:
        result = result @ sigma_anyonic(k)
    return result

# --- Hamiltonian with φ = π/2 ---
phi = np.pi / 2
AB_phase = np.exp(1j * phi)
edges = [(0, 1), (1, 2), (1, 3), (0, 2), (2, 3)]
red_link = (1, 2)
path_dict = {(0, 1): [], (1, 2): [], (1, 3): [], (0, 2): [1], (2, 3): []}

H = np.zeros((16, 16), dtype=complex)
for (i, j) in edges:
    path = path_dict[(i, j)]
    t = AB_phase if (i, j) == red_link or (j, i) == red_link else 1.0
    term = SP(i) @ jw_string(path) @ SM(j)
    H += t * term + np.conj(t) * term.conj().T

# Add density-density interactions
for i in range(4):
    for j in range(i + 1, 4):
        H += number_op(i) @ number_op(j)

# --- Define time evolution ---
T = 1.0
U_target = expm(-1j * H * T)

# --- Native gate tiles ---
def Rx(k, theta):
    ops = [I] * 4
    ops[k] = expm(-1j * theta * X / 2)
    return kron_n(*ops)

def Rz(k, theta):
    ops = [I] * 4
    ops[k] = expm(-1j * theta * Z / 2)
    return kron_n(*ops)

def MS(i, j, theta):
    ops = [I] * 4
    ops[i] = X
    ops[j] = X
    XX = kron_n(*ops)
    return expm(-1j * theta * XX)

def ZZ(i, j, theta):
    # exp(-i theta Z_i Z_j)
    ops = [I] * 4
    ops[i] = Z
    ops[j] = Z
    ZZ_op = kron_n(*ops)
    return expm(-1j * theta * ZZ_op)

# --- Build fixed tile sequence ---
tile_sequence = [
    Rx(0, np.pi / 2),
    MS(0, 1, np.pi / 4),
    Rz(1, np.pi / 2),
    MS(2, 3, np.pi / 4),
    Rx(3, np.pi / 2),
]

U_trial = np.eye(16, dtype=complex)
for gate in tile_sequence:
    U_trial = gate @ U_trial

# --- Compute fidelity ---
fidelity = np.abs(np.trace(U_trial.conj().T @ U_target))**2 / 16**2
print("Fidelity:", fidelity)


Fidelity: 0.006183149715263858


In [6]:
import numpy as np
from scipy.linalg import expm
from scipy.optimize import minimize
from itertools import product
from random import sample

# --- Pauli matrices ---
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# --- Utilities ---
def kron_n(a, b, c, d):
    return np.kron(np.kron(np.kron(a, b), c), d)

n_qubits = 4
dim = 2 ** n_qubits

# --- Target unitary (set this yourself if not already available) ---
# Example placeholder:
# U_target_phi = expm(-1j * H * t)
U_target_phi = U_target 

# --- Native gate definitions ---
def Rx(k, theta):
    ops = [I] * 4
    ops[k] = expm(-1j * theta * X / 2)
    return kron_n(*ops)

def Rz(k, theta):
    ops = [I] * 4
    ops[k] = expm(-1j * theta * Z / 2)
    return kron_n(*ops)

def MS(i, j, theta):
    ops = [I] * 4
    ops[i] = X
    ops[j] = X
    XX = kron_n(*ops)
    return expm(-1j * theta * XX)

def ZZ(i, j, theta):
    ops = [I] * 4
    ops[i] = Z
    ops[j] = Z
    ZZ_op = kron_n(*ops)
    return expm(-1j * theta * ZZ_op)

# --- Tile templates: each is a function(theta) -> list of gates ---
param_tile_templates = [
    lambda theta: [MS(0, 1, theta)],
    lambda theta: [Rx(0, -np.pi / 2), Rx(1, -np.pi / 2), MS(0, 1, theta), Rx(0, np.pi / 2), Rx(1, np.pi / 2)],
    lambda theta: [Rx(2, -np.pi / 2), MS(1, 2, theta), Rx(2, np.pi / 2)],
    lambda theta: [Rx(1, -np.pi / 2), MS(1, 2, theta), Rx(1, np.pi / 2)],
    lambda theta: [MS(1, 3, theta)],
    lambda theta: [Rx(1, -np.pi / 2), Rx(3, -np.pi / 2), MS(1, 3, theta), Rx(1, np.pi / 2), Rx(3, np.pi / 2)],
    lambda theta: [MS(0, 2, theta)],
    lambda theta: [Rx(0, -np.pi / 2), Rx(2, -np.pi / 2), MS(0, 2, theta), Rx(0, np.pi / 2), Rx(2, np.pi / 2)],
    lambda theta: [MS(2, 3, theta)],
    lambda theta: [Rx(2, -np.pi / 2), Rx(3, -np.pi / 2), MS(2, 3, theta), Rx(2, np.pi / 2), Rx(3, np.pi / 2)],
    lambda theta: [Rz(0, theta)],
    lambda theta: [Rz(1, theta)],
    lambda theta: [ZZ(0, 1, theta)],
    lambda theta: [ZZ(0, 2, theta)],
    lambda theta: [ZZ(0, 3, theta)],
    lambda theta: [ZZ(1, 2, theta)],
    lambda theta: [ZZ(1, 3, theta)],
    lambda theta: [ZZ(2, 3, theta)],
]

# --- Joint tile search + optimization ---
tile_count = 10
num_trials = 10000  # increase if desired

best_joint_fidelity = 0
best_joint_tile_indices = None
best_joint_params = None

for trial in range(num_trials):
    tile_indices = sample(range(len(param_tile_templates)), k=tile_count)
    init_params = [np.pi / 4] * tile_count

    def fidelity_loss_joint(params):
        U_trial = np.eye(dim, dtype=complex)
        for i, tile_idx in enumerate(tile_indices):
            tile_fn = param_tile_templates[tile_idx]
            tile_gates = tile_fn(params[i])
            for gate in tile_gates:
                U_trial = gate @ U_trial
        fidelity = np.abs(np.trace(U_trial.conj().T @ U_target_phi))**2 / dim**2
        return 1 - fidelity

    result = minimize(fidelity_loss_joint, init_params, method='Nelder-Mead', options={'maxiter': 300})
    fidelity = 1 - result.fun

    if fidelity > best_joint_fidelity:
        best_joint_fidelity = fidelity
        best_joint_tile_indices = tile_indices
        best_joint_params = result.x

# --- Results ---
print("Best Fidelity:", best_joint_fidelity)
print("Best Tile Indices:", best_joint_tile_indices)
print("Optimized Parameters:", best_joint_params)


Best Fidelity: 0.15594950429140975
Best Tile Indices: [1, 7, 8, 10, 11, 15, 16, 5, 2, 0]
Optimized Parameters: [0.48333717 0.30214509 0.78524696 1.7365293  1.69426544 0.82717024
 0.74220032 0.83614784 0.09573054 0.60283731]
